<a href="https://www.kaggle.com/code/nihxlt/forecast-matrix?scriptVersionId=219416245" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


**<h1 style="color : red;">Who are you? and why are you here?!</h1>**

👨🏻‍💻🔫🐙
-- Authored by
<details>
<summary id="name">
    <a href="#name">Nihal Thakur</a></summary>
    <ul>
        <li><a href="https://github.com/naxl">Github</a></li>
        <li><a href="https://www.kaggle.com/nihxlt">Kaggle</a></li>
    </ul>
</details>

In an unexpected turn of events, you've been enlisted as the chief **"Weather Whisperer"** for the elite **Umbrella Society Summit** set to take place in the fickle climate of **Binghamton, NY**. Your role? To scrape not just any data, but the lifeblood of their entire event—real-time weather insights from this page. The summit’s success hinges on your ability to extract current conditions (humidity, wind speed, temperature — everything down to the UV index), the ever-changing **7-day forecast**, and the detailed table of meteorological minutiae like dew points, visibility, and precipitation probabilities. Yes, even the chance of a rogue sunbeam must be documented!

Your weather-hungry clientele demands perfection. You will scrape the **current conditions** section, where the fate of their outdoor meetings lies. Then, delve into the **forecast table** that dictates whether sunglasses or umbrellas will dominate the day. Lastly, gather all the juicy **details from the in-depth weather table** (hourly temperatures, wind patterns, and those ever-so-important rain chances). Their tea parties and pontifications about cloud formations rest on your data scraping prowess. So, will you rise to the challenge and save the summit, or will their umbrellas be left **high** and **dry**?

**<h3 style="color : blue;">Let's import all the essentials!🌸🐼</h3>**

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

**<h3 style="color : blue;">Fetching the webpage and creating Soup!✅</h3>**

In [2]:
url = "https://forecast.weather.gov/MapClick.php?lat=42.9371&lon=-75.6107"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')


<hr style = "border : 3px solid #E5E1DA; border-radius: 5px">

**Extracting current conditions!**

In [3]:
curr_con = soup.find(class_= "myforecast-current").get_text().strip()
temp_f = soup.find(class_= "myforecast-current-lrg").get_text().strip()
temp_c = soup.find(class_= "myforecast-current-sm").get_text().strip()

df_tod =pd.DataFrame([curr_con,temp_f,temp_c],
                     index = ['Weather', 'Temparture °F', 'Temparture °C'])
df_tod.columns = ['']
df_tod

,
Weather,
Temparture °F,9°F
Temparture °C,-13°C


In [4]:
curr_con_table = soup.find(id="current_conditions_detail").find_all('td')
curr_dict = {}
for i in range(0,len(curr_con_table),2):
    curr_dict[curr_con_table[i].get_text().strip()] = \
    curr_con_table[i+1].get_text().strip()
df_con = pd.DataFrame(curr_dict, index = [''])
df_con.T

,
Humidity,85%
Wind Speed,Calm
Barometer,30.04 in
Dewpoint,5°F (-15°C)
Visibility,10.00 mi
Last update,26 Jan 10:15 pm EST


<hr style = "border : 3px solid #E5E1DA; border-radius: 5px">

**Extracting extended forecast for city!**

In [5]:
curr_ex = [(i.find(class_ = 'period-name').get_text().strip(),
           i.find(class_ = 'temp').get_text().strip(),
           i.find(class_ = 'short-desc').get_text().strip())
           for i in soup.find_all(class_="tombstone-container")]
arr_curr_ex = np.array(curr_ex)
df_curr_ex = pd.DataFrame(arr_curr_ex)
df_curr_ex.columns = ['Period','Temperature','Description']
df_curr_ex.set_index('Period')

,Temperature,Description
Period,,
Tonight,Low: 16 °F,Partly Cloudy
Monday,High: 33 °F,Sunny
Monday Night,Low: 15 °F,Snow ShowersLikely andAreas BlowingSnow
Tuesday,High: 20 °F,Snow Showersand AreasBlowing Snowthen MostlyCl...
Tuesday Night,Low: 13 °F,Chance Snowthen SnowLikely
Wednesday,High: 35 °F,Snow
Wednesday Night,Low: 9 °F,Snow ShowersLikely
Thursday,High: 25 °F,Partly Sunnythen ChanceSnow Showers
Thursday Night,Low: 18 °F,Chance SnowShowers


<hr style = "border : 3px solid #E5E1DA; border-radius: 5px">

**Extracting detailed forecast!**

In [6]:
det_for = [(i.find(class_ = 'forecast-label').get_text().strip(),
           i.find(class_ = 'forecast-text').get_text().strip())
           for i in soup.find_all(class_="row-forecast")]
df_for = pd.DataFrame(det_for)
df_for.columns = ['Period', '']
df_for.set_index('Period')

,
Period,
Tonight,"Partly cloudy, with a low around 16. Southwest..."
Monday,"Sunny, with a high near 33. Southwest wind 11 ..."
Monday Night,"Snow showers likely, mainly after 4am. Areas ..."
Tuesday,"Snow showers, mainly before 7am. Areas of blo..."
Tuesday Night,"Snow likely, mainly after 1am. Mostly cloudy,..."
Wednesday,Snow. High near 35. Chance of precipitation i...
Wednesday Night,"Snow showers likely, mainly before 1am. Mostl..."
Thursday,A chance of snow showers after 1pm. Mostly cl...
Thursday Night,"A chance of snow showers. Mostly cloudy, with..."


<hr style = "border : 3px solid #E5E1DA; border-radius: 5px">